<a href="https://colab.research.google.com/github/nhanguyenai/HomeWork-Predict/blob/main/FaceShapeRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split

IMG_SIZE = 128

def load_and_preprocess_data(data_dir):
    images = []
    labels = []
    face_dir = os.path.join(data_dir, 'faces')
    non_face_dir = os.path.join(data_dir, 'non_faces')

    for img_file in os.listdir(face_dir):
        try:
            img_path = os.path.join(face_dir, img_file)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            img = img / 255.0
            images.append(img)
            labels.append(1)
        except Exception as e:
            print(f"Lỗi khi đọc ảnh: {img_path} - {e}")

    for img_file in os.listdir(non_face_dir):
        try:
            img_path = os.path.join(non_face_dir, img_file)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            img = img / 255.0
            images.append(img)
            labels.append(0)
        except Exception as e:
            print(f"Lỗi khi đọc ảnh: {img_path} - {e}")

    return np.array(images), np.array(labels)

base_dir = '/content/drive/MyDrive/faceshapedatasets'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
valid_dir = os.path.join(base_dir, 'valid')

train_images, train_labels = load_and_preprocess_data(train_dir)
test_images, test_labels = load_and_preprocess_data(test_dir)
valid_images, valid_labels = load_and_preprocess_data(valid_dir)

print("Kích thước tập huấn luyện:", train_images.shape, train_labels.shape)
print("Kích thước tập kiểm tra:", test_images.shape, test_labels.shape)
print("Kích thước tập xác thực:", valid_images.shape, valid_labels.shape)

Kích thước tập huấn luyện: (818, 128, 128, 3) (818,)
Kích thước tập kiểm tra: (107, 128, 128, 3) (107,)
Kích thước tập xác thực: (86, 128, 128, 3) (86,)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_model(input_shape):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid') # Sử dụng sigmoid cho phân loại nhị phân (có/không phải khuôn mặt)
    ])
    return model

input_shape = (IMG_SIZE, IMG_SIZE, 3) # 3 kênh màu (RGB)
model = build_model(input_shape)

# Biên dịch mô hình
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
epochs = 50
batch_size = 32

history = model.fit(
    train_images, train_labels,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(valid_images, valid_labels)
)

Epoch 1/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 0.8560 - loss: 0.4435 - val_accuracy: 0.3488 - val_loss: 0.8812
Epoch 2/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.9326 - loss: 0.1903 - val_accuracy: 0.9651 - val_loss: 0.1270
Epoch 3/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.9769 - loss: 0.0568 - val_accuracy: 0.9535 - val_loss: 0.1190
Epoch 4/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.9982 - loss: 0.0145 - val_accuracy: 0.9302 - val_loss: 0.1551
Epoch 5/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.9996 - loss: 0.0169 - val_accuracy: 0.8953 - val_loss: 0.2062
Epoch 6/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 1.0000 - loss: 0.0049 - val_accuracy: 0.9767 - val_loss: 0.0722
Epoch 7/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step - accuracy: 1.0000 - loss: 7.9601e-04 - val_accuracy: 0.9767 - val_loss: 0.0573
Epoch 8/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 1.0000 - loss: 7.3212e-04 - val_accuracy: 0.9767 - v

In [ ]:
model.save('/content/drive/MyDrive/FaceRecognition.h5')
print("Mô hình đã được lưu vào /content/drive/MyDrive/my_trained_model.h5")

Mô hình đã được lưu vào /content/drive/MyDrive/my_trained_model.h5


In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

IMG_SIZE = 128

def predict_image(img_path, model):
    try:
        img = cv2.imread(img_path)
        img_resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img_normalized = img_resized / 255.0
        img_expanded = np.expand_dims(img_normalized, axis=0)
        prediction = model.predict(img)
        return prediction[0][0], img
    except Exception as e:
        print(f"Lỗi khi xử lý ảnh: {img_path} - {e}")
        return None, None

MODEL_PATH = '/content/face_recognition_model.h5'

image_to_test = 'test_image.jpg'

try:
    model = tf.keras.models.load_model(MODEL_PATH)
    print(f"Đã tải mô hình thành công từ: {MODEL_PATH}")
except Exception as e:
    print(f"Lỗi khi tải mô hình từ: {MODEL_PATH} - {e}")
    exit()

prediction_value, test_image = predict_image(image_to_test, model)

if prediction_value is not None and test_image is not None:
    plt.imshow(cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB))
    plt.title(f"Dự đoán: {'Khuôn mặt' if prediction_value > 0.5 else 'Không phải khuôn mặt'} ({prediction_value:.4f})")
    plt.axis('off')
    plt.show()

    if prediction_value > 0.5:
        print(f"Ảnh '{image_to_test}' được dự đoán là khuôn mặt người (độ tin cậy: {prediction_value:.4f})")
    else:
        print(f"Ảnh '{image_to_test}' được dự đoán là không phải khuôn mặt người (độ tin cậy: {1 - prediction_value:.4f})")
else:
    print("Không thể thực hiện dự đoán.")

Lỗi khi tải mô hình từ: /content/face_recognition_model.h5 - Unable to synchronously open file (truncated file: eof = 13631488, sblock->base_addr = 0, stored_eof = 39710776)
Lỗi khi xử lý ảnh: test_image.jpg - OpenCV(4.11.0) /io/opencv/modules/imgproc/src/resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Không thể thực hiện dự đoán.


In [3]:
import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

IMG_SIZE = 128

def is_face(img_path, model_path):

    try:
        model = tf.keras.models.load_model(model_path)

        img = cv2.imread(img_path)
        if img is None:
            print(f"Lỗi: Không thể đọc ảnh từ đường dẫn: {img_path}")
            return False, None, None

        img_resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img_normalized = img_resized / 255.0
        img_expanded = np.expand_dims(img_normalized, axis=0)

        prediction = model.predict(img_expanded)[0][0]

        is_face_bool = prediction > 0.5

        return is_face_bool, prediction, img

    except Exception as e:
        print(f"Lỗi khi xử lý ảnh hoặc mô hình: {e}")
        return False, None, None

def display_prediction(img, is_face, confidence):
    """Hiển thị ảnh với kết quả dự đoán."""
    if img is not None:
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        title = f"Dự đoán: {'Khuôn mặt' if is_face else 'Không phải khuôn mặt'}\nĐộ tin cậy: {confidence:.4f}"
        plt.title(title)
        plt.axis('off')
        plt.show()

if __name__ == '__main__':
    model_path = '/content/face_recognition_model.h5'

    image_to_check = '/content/face2.jpg'

    is_face_result, confidence_score, displayed_image = is_face(image_to_check, model_path)

    if confidence_score is not None:
        print(f"Ảnh '{image_to_check}' được dự đoán là {'khuôn mặt' if is_face_result else 'không phải khuôn mặt'} với độ tin cậy {confidence_score:.4f}")
        display_prediction(displayed_image, is_face_result, confidence_score)
    else:
        print("Không thể thực hiện kiểm tra ảnh.")

Lỗi khi xử lý ảnh hoặc mô hình: Unable to synchronously open file (truncated file: eof = 33554432, sblock->base_addr = 0, stored_eof = 39710776)
Không thể thực hiện kiểm tra ảnh.
